# Heart Failure EDA & Prediction using Logistic Regression, SVM, RandomForest, XGBoost, and Neural Networks with Python 
  
  ## Table of Contents  
* [Introduction](#introduction)
* [Understanding the Dataset](#understand)
* [Exploratory Data Analysis](#EDA)
    - [Categorical Data](#categorical)
    - [Numerical Data](#numerical)
* [Outlier Detection](#outlier)
* [Predictive Analysis](#predictive)
    - [Data Preprocessing](#preprocessing)
    - [Logistic Regression](#lr)
    - [Support Vector Machines](#svm)
    - [RandomForest Classifier](#rf)
    - [XGBoost](#xgb)
    - [FeedForward Neural Networks](#nn)
* [Verdict](#verdict)


**IMPORTANT**  - Some of the code (Gridsearch) have been commented out because it takes a long time to run. You can try those parts by uncommenting the codeblocks.

<a id="introduction"></a>  
## Introduction

##### Importing the dataset

In [1]:
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


heart = pd.read_csv('../input/heart-failure-prediction/heart.csv')
heart.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/heart-failure-prediction/heart.csv'

<a id="understand"></a>  
#### Understanding what variables in the dataset means and their types
  1. Age - Age of the Patient - **Numerical**
  2. Sex - Gender of the Patient - **Categorical**
        * M - Male
        * F - Female
  3. ChestPainType - Floor is made of floor - **Categorical**
        * TA - [Typical Angina](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5680106) - Substernal chest pain precipitated by physical exertion or emotional stress
        * ATA - [ATypical Angina](https://www.ncbi.nlm.nih.gov/medgen/149267) Angina pectoris which does not have associated classical symptoms of chest pain. Symptoms - weakness, nausea, or sweating
        * NAP - [Non-Anginal Chest Pain](https://my.clevelandclinic.org/health/diseases/15851-gerd-non-cardiac-chest-pain) - Pain in the chest that is NOT caused by Heart Disease or Heart Attack
        * ASY - [Asymptomatic](https://www.mayoclinic.org/diseases-conditions/heart-attack/expert-answers/silent-heart-attack/faq-20057777) - No symptoms
  4. RestingBP - [Resting Blood Pressure (mm/Hg)](https://www.medicinenet.com/blood_pressure_chart_reading_by_age/article.htm) - **Numerical**
  5. Cholesterol - [Serum Cholesterol (mm/dl)](https://www.medicalnewstoday.com/articles/321519) - **Numerical**
  6. FastingBS - [Fasting Blood Sugar](https://www.mayoclinic.org/diseases-conditions/diabetes/diagnosis-treatment/drc-20371451) - **Categorical (1: if FastingBS > 120 mg/dl, 0: otherwise)**
  7. RestingECG - Resting ElectroCardiogram Results - **Categorical**
        * Normal - [Normal ECG Reading](https://ecgwaves.com/topic/ecg-normal-p-wave-qrs-complex-st-segment-t-wave-j-point/) 
        * ST - [Abnormality in ST-T Wave Part of ECG](https://www.healio.com/cardiology/learn-the-heart/ecg-review/ecg-interpretation-tutorial/68-causes-of-t-wave-st-segment-abnormalities) 
        * LVH - [Probable or definite Left Ventricular hypertrophy](https://www.healio.com/cardiology/learn-the-heart/ecg-review/ecg-interpretation-tutorial/68-causes-of-t-wave-st-segment-abnormalities) 
  8. MaxHR - Maximum Heart Rate Achieved (60-202) - **Numeric**
  9. ExerciseAngina - [Exercise Induced Angina](https://www.mayoclinic.org/diseases-conditions/angina/symptoms-causes/syc-20369373) - When your Heart wants more blood,but narrowed arteries slow down the blood flow - **Categorical (Yes/No)**
  10. Oldpeak - [ST Depression](https://en.wikipedia.org/wiki/ST_depression) - **Numerical**
  11. ST_Slope - [Slope](https://pubmed.ncbi.nlm.nih.gov/3739881/) of the peak exercise ST Segment - **Categorical**
        * Up - Upward Slope
        * Flat - Slope is zero
        * Down - Downward Slope
  12. HeartDisease - Output Class - **Categorical (1: Heart Disease,0: Normal)**

In [ ]:
#Shape of the dataset
print("Shape of the dataset is: "+str(heart.shape))
#Is there any null values in the data?
print("Amount of null values in data: "+ str(heart.isnull().sum().sum()))

As we can see, there are 918 observations in the dataset. We don't have any missing data.

In [ ]:
#Check balance of the output variables
heart.groupby(['HeartDisease'])['HeartDisease'].count()

We can see that there is a slight imbalance in our dataset. We'll use methods such as balancing the dataset by removing excess observations (Results in loss of data, Using performance metrics such as precision, Recall, F1-Score other than accuracy, etc..

<a id="EDA"></a>  
### Exploratory Data Analysis
<a id="categorical"></a>  
#### Categorical Variables

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.patch.set_facecolor('lightblue')
fig.patch.set_alpha(0.3)

sns.countplot(x = heart['RestingECG'], data = heart, palette='colorblind', ax=axes[0,0],hue = "HeartDisease")
sns.countplot(x = heart['ChestPainType'], data = heart, palette='colorblind', ax=axes[0,1],hue = "HeartDisease")

sns.countplot(x = heart['ExerciseAngina'], data = heart, palette='colorblind', ax=axes[0,2],hue = "HeartDisease")
sns.countplot(x = heart['FastingBS'], data = heart, palette='colorblind', ax=axes[1,0],hue = "HeartDisease")
sns.countplot(x = heart['Sex'], data = heart, palette='colorblind', ax=axes[1,1],hue = "HeartDisease")
sns.countplot(x = heart['ST_Slope'], data = heart, palette='colorblind', ax=axes[1,2],hue = "HeartDisease")
plt.suptitle('Categorical Variables EDA', fontsize=25)
plt.show()

##### Decoding these plots can be somewhat tricky.  
We had more data for people with heart disease than not. So, small differences between two hues are ignored. After carefully analysing these plots, we can get following insights,
* People with ST-T wave abnormalities are more likely to have a heart disease.
* There is an alarmingly high amount of patients with Asymptomatic heart diseases.
* People who get Exercise induced Angina are more likely to have a heart disease.
* People with high fasting blood sugar levels are more likely to have a heart disease.
* Men are more likely to get heart diseases than women.
* People with flat or down ST_Slope are more likely to have a heart disease.

<a id="numerical"></a>  
#### Numerical Variables

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(17, 10))
fig.patch.set_facecolor('lightgreen')
fig.patch.set_alpha(0.2)

sns.histplot(data = heart,x = heart['Age'],hue = "HeartDisease",element="poly",ax=axes[0,0])
sns.histplot(data = heart,x = heart['RestingBP'],hue = "HeartDisease",element="poly",ax=axes[0,1])
sns.histplot(data = heart,x = heart['Cholesterol'],hue = "HeartDisease",element="poly",ax=axes[0,2])
sns.histplot(data = heart,x = heart['MaxHR'],hue = "HeartDisease",element="poly",ax=axes[1,0])
sns.histplot(data = heart,x = heart['Oldpeak'],hue = "HeartDisease",element="poly",ax=axes[1,1])
fig.delaxes(axes[1][2])
plt.suptitle('Numerical Variables EDA', fontsize=25)
plt.show()

We can get following insights from Numerical variables,
* People tend to get more susceptible to heart diseases as they get older.
* Resting blood pressures of people with and without heart diseases are kinda similar. Maybe Resting BP is slightly higher for people with heart disease.
* **There are unusually high amount of people with zero cholesterol level. This might be a error in data. We have to see about that while considering variables for Machine Learning.**
* Maximum heart rate is lower (Highest count is around 120) for most people with heart diseases. Healthy (No heart Disease) people tend to have higher Maximum Heart Rate.
* ST wave depression peak value is generally higher for people with heart disease.

In [ ]:
# Correlation heatmap for Numerical and Binary Variables
sns.heatmap(heart.corr(), annot=True, cmap='mako')
plt.show()

<a id="outlier"></a>  
#### Outlier Detection
Now, We can try to find Outliers from Numerical Variables and determine whether they should be removed or not.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
sns.boxplot(x=heart['RestingBP'],ax=axes[0,0])
sns.boxplot(x=heart['Cholesterol'],ax=axes[0,1])
sns.boxplot(x=heart['MaxHR'],ax=axes[1,0])
sns.boxplot(x=heart['Oldpeak'],ax=axes[1,1])
plt.suptitle("Detecting Outliers",fontsize = 15)
plt.show()

**Resting Blood Pressure**

In [ ]:
#It is impossible to have a resting heart bloop pressure of zero.
print((heart['RestingBP']==0).sum())

#We need to impute this datapoint (not the whole row). I will do this with the median Resting blood pressure
for i in heart['RestingBP']:
    if i == 0:
        heart['RestingBP'] = heart['RestingBP'].replace(i, heart['RestingBP'].median())

#Checking if high BP has a correlation with the outcome
qHigh = heart['RestingBP'].quantile(0.80)
print("High Resting BP on Heart Disease: ")
print(heart[(heart['RestingBP'] >= qHigh)].value_counts(heart['HeartDisease']))

#We can see that there is a increase in heart disease with high RestingBP. So we will not change this outliers.

**Cholesterol**

In [ ]:
#Low cholesterol levels seems suspicious. But there are many datapoints.
print((heart['Cholesterol']==0).sum())
#Check correlation of heart disease with zero cholesterol level.
print("Zero Cholesterol on Heart Disease: ")
print(heart[(heart['Cholesterol'] == 0)].value_counts(heart['HeartDisease']))

#BUTTT, WE CAN SEE THAT IT HEART DISEASE IS COMMON WITH VERY LOW CHOLESTEROL LEVELS SO WE will keep the data.


**Maximum Heart Rate**

In [ ]:
#There are some rows with very low maximum heart rates. Lets see how it affects the Heart disease
print("Low Maximum Heart Rate on Heart Disease: ")
print(heart[(heart['MaxHR'] <= 75)].value_counts(heart['HeartDisease']))
#WE CAN SEE THAT LOW MaxHR is ASSOCIATED WITH Heart Disease. So we will not touch this datapoints

**OldPeak**

In [ ]:
#There are some rows with very low and very high Old peaks. Lets see how it affects the Heart diseas
print("Low Oldpeak on Heart Disease: ")
print(heart[(heart['Oldpeak'] <= -1)].value_counts(heart['HeartDisease']))

print("High Oldpeak on Heart Disease: ")
print(heart[(heart['Oldpeak'] >= 3.5)].value_counts(heart['HeartDisease']))

# We can see that very low and very high old peaks are associated with Heart Disease. 

<a id="predictive"></a>  
### Predictive Analysis

<a id="preprocessing"></a>  
#### Data Preprocessing

In [ ]:
#Setting up X and y Variables
X = heart.iloc[:,0:-1] # All columns except HeartDisease Columnn
y = heart.iloc[:,-1] # HeartDisease Column

In [ ]:
#One Hot Encoding using Pandas Get Dummies function
X = pd.get_dummies(X, columns=['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope'], 
                   drop_first=True) #We dont need to encode Fasting BS because it's already in a 0 or 1 format
#drop_first = true to avoid dummy variable trap. We are removing one encoded column from every categorical variable
X.head()

In [ ]:
#Splitting data into training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

<a id="lr"></a>  
#### Logistic Regression

In [ ]:
#For Logistics Regression we dont need to scale data manually
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver='liblinear',random_state =0)
#If you dont include solver='liblinear', you'll get an error for some reason (This doesnt happen when i do it offline)
LR.fit(X_train,y_train)
y_pred = LR.predict(X_test)

In [ ]:
#Confusion Matrix and Accuracy
from sklearn.metrics import accuracy_score, f1_score,classification_report
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm, annot=True)
accuracy = round(accuracy_score(y_test,y_pred),4)
print("Accuracy: ",accuracy)
f1 = f1_score(y_test,y_pred)
print("F1 Score:\n ",f1)
report = classification_report(y_test,y_pred)
print("Classification Report:\n ",report)

In [ ]:
#Using KFold Cross Validation
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, random_state=1, shuffle=True)
from sklearn.model_selection import cross_validate
scores = cross_validate(LogisticRegression(solver='liblinear',random_state =0), 
                        X, y, cv=cv,scoring = ['accuracy','f1_macro','precision','recall'])

print('Accuracy: %.4f (std: %.3f)' % (np.mean(scores['test_accuracy']), np.std(scores['test_accuracy'])))
print('F1 Score : %.4f' % (np.mean(scores['test_f1_macro'])))
print('Precision : %.4f' % (np.mean(scores['test_precision'])))
print('Recall : %.4f' % (np.mean(scores['test_recall'])))

<a id="svm"></a>  
#### Support Vector Machines (Classification)

In [ ]:
# Feature Scaling is needed
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() 
'''
LOG - #We will only scale numerical X values and concat that with categorical part (Excluding FastingBS im too lazy lol)
Turns out this don't work very well..., Scaling all variables equaly performs better (Done below)
This vs Equal Scaling Results are as below(F1 Score),
SVC (grid search) - 0.8424 vs 0.8533
RandomForest (grid_search) - 0.8501 vs 0.8501
NN (grid_search) - It changes a lot LMAO
X_scaled = pd.concat([pd.DataFrame(sc.fit_transform(X.iloc[:,0:6]),columns = X.columns[0:6]),X.iloc[:,6:]],axis = 1)
'''
X_scaled = pd.DataFrame(sc.fit_transform(X),columns = X.columns)
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=0)
X_scaled.head()

In [ ]:
#Simple Model
from sklearn.svm import SVC
svm = SVC(random_state=1)
svm.fit(X_train_scaled,y_train)
y_pred = svm.predict(X_test_scaled)

cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm, annot=True)
accuracy = round(accuracy_score(y_test,y_pred),4)
print("Accuracy: ",accuracy)
f1 = f1_score(y_test,y_pred)
print("F1 Score:\n ",f1)
report = classification_report(y_test,y_pred)
print("Classification Report:\n ",report)

In [ ]:
#It's already better than  Logistic Regression even with stock parameters (C=1,rbf,degree = 3,gamma = scale)

#Now I will Optimize Hyperparameters
from sklearn.model_selection import GridSearchCV
'''
params = [ {'C':[1, 10, 100,1000], 'kernel':['linear']},
               {'C':[1, 10, 100,1000], 'kernel':['rbf'], 'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,'scale']},
               {'C':[1, 10, 100,1000], 'kernel':['poly'], 'degree': [2,3,4,5] ,'gamma':[0.01,0.02,0.03,0.04,0.05,'scale']},
                {'C':[1, 10, 100,1000], 'kernel':['sigmoid']}
              ]
grid = GridSearchCV(estimator = svm,  
                           param_grid = params,
                           scoring = 'f1', #F1 because false positives and false negatives are crucial
                           cv = 5,
                           verbose=0)
grid.fit(X_scaled,y)
'''

In [ ]:
#y_pred = grid.predict(X_test_scaled)
#cm = confusion_matrix(y_test,y_pred)
#accuracy = round(accuracy_score(y_test,y_pred),4)
#print("Accuracy: ",accuracy)# 0.8913
#sns.heatmap(cm,annot=True,fmt = 'g')
#Getting Best Scores and Best Parameters
#print("Best Score: ", grid.best_score_) #F1 - 0.8529593322446811 , Accuracy  - 0.8913
#print("Best Parameters : ", grid.best_params_) #  {'C': 1, 'degree': 5, 'gamma': 'scale', 'kernel': 'poly'}

In [ ]:
#Creating a model according to best parameters 
svm = SVC(C = 1,degree = 5,kernel = 'poly',gamma = 'scale',random_state=1)
svm.fit(X_train_scaled,y_train)
y_pred = svm.predict(X_test_scaled)

cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm, annot=True)
accuracy = round(accuracy_score(y_test,y_pred),4)
print("Accuracy: ",accuracy) # Accuracy & F1 we get here is lower than grid search model probably due to less avalable data
f1 = f1_score(y_test,y_pred)
print("F1 Score:\n ",f1)
report = classification_report(y_test,y_pred)
print("Classification Report:\n ",report)

<a id="rf"></a>  
#### Random Forest Classifier

In [ ]:
#I will start with Grid Search (Takes so long to run)
from sklearn.ensemble import RandomForestClassifier 
'''
RF = RandomForestClassifier(random_state = 0)
params = { 
    'n_estimators': [10,20,30,50,100,200,300,400,500,750,900],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8,None],
    'criterion' :['gini', 'entropy']
}
grid_RF = GridSearchCV(estimator = RF,  
                           param_grid = params,
                           scoring = 'f1', #F1 because false positives and false negatives are crucial
                           cv = 5,
                           verbose=0)
grid_RF.fit(X_scaled,y)

y_pred = grid_RF.predict(X_test_scaled)
cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm,annot=True)
'''

In [ ]:
#Getting Best Scores and Best Parameters
#print("Best Score: ", grid_RF.best_score_) #0.8500657209209763
#accuracy = round(accuracy_score(y_test,y_pred),4) #0.8478
#print("Accuracy: ",accuracy)# 0.8913
#print("Best Parameters : ", grid_RF.best_params_) #  {'criterion': 'entropy', 'max_depth': 4, 'max_features': 'auto', 'n_estimators': 20

In [ ]:

#Creating a model according to best parameters
RF = RandomForestClassifier(criterion = 'entropy',max_depth = 4,max_features = 'auto',n_estimators = 20,random_state=0)
RF.fit(X_train_scaled,y_train)
y_pred = RF.predict(X_test_scaled)

cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm, annot=True)
accuracy = round(accuracy_score(y_test,y_pred),4)
print("Accuracy: ",accuracy)
f1 = f1_score(y_test,y_pred)
print("F1 Score:\n ",f1)
report = classification_report(y_test,y_pred)
print("Classification Report:\n ",report)

<a id="xgb"></a>  
#### XGBoost Classifier

In [ ]:
#I will start with Grid Search (Takes so long to run)

from xgboost import XGBClassifier
'''
xgb = XGBClassifier(random_state = 0,verbosity = 0,use_label_encoder = False)
#{'booster':['gblinear'],'lambda':[0.01, 0.1, 0.5],'updater':['shotgun','coord_descent'],
               # 'feature_selector':['cyclic','shuffle'],'top_k':[0,1,2,3,4,5]},
params = [ {'learning_rate':[0.01, 0.1, 0.2,0.3,0.4], 'booster':['gbtree'], 'min_split_loss':[0,0.01,0.1],
                   'max_depth':[3,4,5,6,7,8,9]},
               {'learning_rate':[0.01, 0.1, 0.2,0.3,0.4], 'booster':['dart'], 'min_split_loss':[0,0.01,0.1],
                   'max_depth':[3,4,5,6,7,8,9]}
              ]
grid_xg = GridSearchCV(estimator = xgb,  
                           param_grid = params,
                           scoring = 'f1', #F1 because false positives and false negatives are crucial
                           cv = 5,
                           verbose=0)
grid_xg.fit(X_scaled,y)

y_pred = grid_xg.predict(X_test_scaled)
cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm,annot=True,fmt = 'g')
'''

In [ ]:
#Getting Best Scores and Best Parameters
#print("Best Score: ", grid_xg.best_score_) #0.839683048346517
#accuracy = round(accuracy_score(y_test,y_pred),4) #0.9076
#print("Accuracy: ",accuracy)# 0.9076
#print("Best Parameters : ", grid_xg.best_params_) #  {'booster': 'gbtree', 'learning_rate': 0.1, 'max_depth': 3, 'min_split_loss': 0}

In [ ]:
#Creating a model according to best parameters
xgb = XGBClassifier(booster = 'gbtree',learning_rate = 0.1,max_depth = 3,
                             min_split_loss = 0,random_state=0,verbosity = 0,use_label_encoder = False)
xgb.fit(X_train_scaled,y_train)
y_pred = xgb.predict(X_test_scaled)

cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm, annot=True)
accuracy = round(accuracy_score(y_test,y_pred),4)
print("Accuracy: ",accuracy)
f1 = f1_score(y_test,y_pred)
print("F1 Score:\n ",f1)
report = classification_report(y_test,y_pred)
print("Classification Report:\n ",report)

<a id="nn"></a>  
### Feedforward Artificial Neural Networks

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras import metrics
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import callbacks

##### Simple Model

In [ ]:
#Simple Model
simpleModel = Sequential(
    [
        Dense(units = 15, activation="relu",input_dim = 15),
        Dense(7, activation="relu"),
        Dense(4,activation = 'relu'),
        Dense(1,activation = 'sigmoid') #output layer
    ]
)
simpleModel.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = ['binary_accuracy'])

simpleHistory = simpleModel.fit(X_train_scaled,y_train,epochs = 1000 ,batch_size = 100,verbose = 0)

In [ ]:
# summarize history for accuracy
plt.plot(simpleHistory.history['binary_accuracy'])
#plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(simpleHistory.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc='upper left')
plt.show()

Looks like we need more regularization.

In [ ]:
y_pred = simpleModel.predict(X_test_scaled)
#print(y_pred)
y_pred = y_pred>0.1
#print(y_pred)
cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm, annot=True)
accuracy = round(accuracy_score(y_test,y_pred),4)
print("Accuracy: ",accuracy)
f1 = f1_score(y_test,y_pred)
print("F1 Score:\n ",f1)
report = classification_report(y_test,y_pred)
print("Classification Report:\n ",report)

##### Tuned Model

[Swish](https://medium.com/@neuralnets/swish-activation-function-by-google-53e1ea86f820) Activation function performs slightly better than ReLU.

In [ ]:
def baseline_model():
    model = Sequential(
    [
        Dense(units = 15, activation='swish',input_dim = 15),
        Dense(10, activation="swish"),
        Dropout(0.1),
        Dense(7,activation = 'swish'),
        Dropout(0.2),
        Dense(1,activation = 'sigmoid') #output layer
    ]
    )
    model.compile(loss = 'binary_crossentropy',optimizer = 'adam', metrics = metrics.BinaryAccuracy(threshold = 0.6))
    return model

model  = baseline_model()

#Using early stopping to counter overfitting
earlystopping = callbacks.EarlyStopping(monitor='val_loss',
                                        mode='min',
                                        verbose=0,
                                        patience=30) #patience is number of epochs to wait after min monitoring variable

history = model.fit(X_train_scaled,y_train,validation_data = (X_test_scaled,y_test),
                    epochs = 500 ,batch_size = 200,verbose = 0,callbacks =[earlystopping])


In [ ]:
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

y_pred = model.predict(X_test_scaled)
#print(y_pred)

i = 0
maxacc = 0;
bestthres = 0;
while i <1:
    y_pred_thres = y_pred>i
    accuracy  = accuracy_score(y_test, y_pred_thres)
    if maxacc< accuracy:
        maxacc = accuracy
        bestthres = i;
    i=i+0.05
print("Best Accuracy: ",maxacc,"at threshold: ",bestthres)

y_pred = y_pred>bestthres

cm = confusion_matrix(y_test,y_pred)
sns.heatmap(cm, annot=True,fmt='g')
accuracy = round(accuracy_score(y_test,y_pred),4)
print("Accuracy: ",accuracy)
f1 = f1_score(y_test,y_pred)
print("F1 Score:\n ",f1)
report = classification_report(y_test,y_pred)
print("Classification Report:\n ",report)

In [ ]:
#K-Fold Cross Validation
estimator = KerasClassifier(build_fn=baseline_model, epochs=500, batch_size=200,verbose = 0)

kfold = KFold(n_splits=10, shuffle=True)
results = cross_validate(estimator, X_scaled, y, cv=kfold,scoring = ['accuracy','f1_macro','precision','recall'],
                         fit_params={'callbacks':earlystopping})
print('Accuracy: %.4f (std: %.3f)' % (np.mean(results['test_accuracy']), np.std(results['test_accuracy'])))
print('F1 Score : %.4f (std: %.3f)' % (np.mean(results['test_f1_macro']) ,np.std(results['test_f1_macro'])))
print('Precision : %.4f' % (np.mean(results['test_precision'])))
print('Recall : %.4f' % (np.mean(results['test_recall'])))

<a id="verdict"></a>  
### Verdict   
  
  With these results, It's clear that using neural networks might not be the best idea for this dataset. This is probably due to the limited amount of data we have. Most of the models performs pretty similar to each other. XGBoost (Grid Search) acheives the best accuracy but f1 score is not optimal. Feedforward Neural Networks model acheives better F1 Scores.